In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,41497
2,Huelva,Confirmados PDIA 14 días,3083
3,Huelva,Tasa PDIA 14 días,"600,7755714480581"
4,Huelva,Confirmados PDIA 7 días,1434
5,Huelva,Tasa PDIA 7 dias,"279,4395619385389"
6,Huelva,Total Confirmados,41695
7,Huelva,Curados,35135
8,Huelva,Fallecidos,396


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  41497.0


/tmp/ipykernel_3209/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  11594.0


/tmp/ipykernel_3209/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3209/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3209/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3209/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 342 personas en los últimos 7 días 

Un positivo PCR cada 166 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,41497.0,1434.0,3083.0,513170.0,279.439562,600.775571,290.0
Huelva-Costa,24387.0,848.0,1816.0,289548.0,292.870267,627.184439,207.0
Huelva (capital),11594.0,420.0,869.0,143837.0,291.997191,604.156093,101.0
Condado-Campiña,13033.0,480.0,992.0,156231.0,307.237360,634.957211,68.0
Isla Cristina,2806.0,82.0,189.0,21393.0,383.302950,883.466554,26.0
Lepe,2723.0,97.0,198.0,27880.0,347.919656,710.186514,25.0
Ayamonte,1783.0,88.0,160.0,21104.0,416.982563,758.150114,21.0
Sierra de Huelva-Andévalo Central,3692.0,102.0,265.0,67391.0,151.355522,393.227582,17.0
Moguer,1812.0,88.0,176.0,21867.0,402.432890,804.865779,13.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Sanlúcar de Guadiana,30.0,16.0,23.0,403.0,3970.223325,5707.196030,3.0
Cumbres Mayores,120.0,6.0,38.0,1749.0,343.053173,2172.670097,0.0
Marines (Los),14.0,5.0,8.0,399.0,1253.132832,2005.012531,0.0
Cañaveral de León,15.0,3.0,7.0,396.0,757.575758,1767.676768,0.0
Escacena del Campo,201.0,18.0,38.0,2287.0,787.057280,1661.565369,4.0
Corteconcepción,42.0,5.0,8.0,536.0,932.835821,1492.537313,3.0
Nava (La),6.0,1.0,3.0,258.0,387.596899,1162.790698,0.0
Villarrasa,210.0,19.0,24.0,2211.0,859.339665,1085.481682,1.0
Villablanca,247.0,12.0,31.0,2885.0,415.944541,1074.523397,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cumbres Mayores,120.0,6.0,38.0,1749.0,343.053173,2172.670097,0.0,0.157895
Villanueva de los Castillejos,190.0,2.0,10.0,2825.0,70.796460,353.982301,0.0,0.200000
Encinasola,161.0,2.0,9.0,1310.0,152.671756,687.022901,0.0,0.222222
Cerro de Andévalo (El),185.0,2.0,9.0,2327.0,85.947572,386.764074,0.0,0.222222
Gibraleón,863.0,20.0,81.0,12737.0,157.022847,635.942530,0.0,0.246914
Punta Umbría,1167.0,29.0,115.0,15355.0,188.863562,748.941713,5.0,0.252174
Calañas,239.0,4.0,14.0,2768.0,144.508671,505.780347,0.0,0.285714
Minas de Riotinto,165.0,6.0,20.0,3812.0,157.397692,524.658972,2.0,0.300000
Nava (La),6.0,1.0,3.0,258.0,387.596899,1162.790698,0.0,0.333333
